In [1]:
import pandas as pd
df= pd.read_csv(r'C:\Users\user\Desktop\카드 데이터\card_data\2022 정제\creaditcard_P_2022.csv',encoding='cp949')

G_R= [561,5611,56111,56112,56113,56114,56119,562,5621,56211,56212,56219]# General restaurant
R_R= [5619,56191,56192,56193,56194,56199,5622,56220] #resting restaurant
A_F= [551,5511,55111,55112,55113,55119,559,5590,55909] #accommodation facility
R_S = [47,471,4711,47111,47119,4712,47121,47122,47129,4719,
        47190,472,4721,47211,47212,47213,47214,47215,47216,
        47219,4722,47221,47222,474,476,4761,47611,4764,47640,
        477,4771,47711,47712,478,4781,47813,4784,47841,47842,
        4785,47852,4786] #retail store
Rent_S = [6911,69110,69190,692,6921,69210]#rent
Sport=[9022,90221,90222,9023,90231,90232,9029,90290,9112,91121,91122,9113,91133,91134,91135,91136,91139,9119,91191,91199,912,9121,
        91210,9122,91221,91222,91223,91229,9123,91231,91239,901,9011,90110,9012,90122,90123,9013,90131,9019]

연령대별 분류

In [2]:
def AGE(data,input):
    data1=data[(data["M_DONG_NM"]=="%s"%input)]
    domestic=data1[data1["TOUR_GB"] == "도민"].index
    data2=data1.drop(domestic)
    data2=data2.iloc[:,2:]
    data3=data2.groupby(["TA_YM","M_DONG_NM","AGE_GB",'IDCS_CD']).sum()
    data3=data3.reset_index()
    data3=data3[["TA_YM","M_DONG_NM","AGE_GB",'IDCS_CD',"AMT_CORR"]]
    data5=data3.groupby(['TA_YM',"AGE_GB"]).sum()
    data5=data5.reset_index()
    data5=data5[['TA_YM','AGE_GB',"AMT_CORR"]]
    AG_2=data5[(data5["AGE_GB"]==1)|(data5["AGE_GB"]==2)].groupby(['TA_YM']).sum()
    AG_6=data5[(data5["AGE_GB"]==6)|(data5["AGE_GB"]==7)|(data5["AGE_GB"]==8)].groupby(['TA_YM']).sum()
    
    AG_2=AG_2.reset_index()
    AG_6=AG_6.reset_index()

    dfl=data5["TA_YM"].drop_duplicates() ## 중복행 처리
    fin_AGE=pd.DataFrame()
    fin_AGE["연도"]=dfl

    for i in range(1,9):
        if i ==1 or i==2:
            fin_AGE["20대 이하"] =0
            #fin_AGE["20대 이하횟수"]=0
            fin_AGE["20대 이하"] =AG_2.iloc[:,2].values
            #fin_AGE["20대 이하횟수"] =AG_2.iloc[:,3].values
        elif i == 6 or i== 7 or i== 8:
            fin_AGE["60대 이상"] =0
            #fin_AGE["60대 이상횟수"] =0
            fin_AGE["60대 이상"] =AG_6.iloc[:,2].values
            #fin_AGE["60대 이상횟수"] =AG_6.iloc[:,3].values
        else:
            fin_AGE["%d0대" % i]=0
            #fin_AGE["%d0대횟수" % i]=0
            fin_AGE["%i0대" %i]=data5.iloc[data5[data5["AGE_GB"] == i].index,2].values
            #fin_AGE["%i0대횟수" %i]=data5.iloc[data5[data5["AGE_GB"] == i].index,3].values
            
    fin_AGE.to_csv('C:\\Users\\user\\Desktop\\%s 연령대별.csv'%input, encoding='cp949')


AGE(df,input="애월읍")

업종별 분류

In [3]:
def industry(data,AGE1,AGE2,AGE3,input):
    data1=data[(data["M_DONG_NM"]=="%s"%input)&((data["AGE_GB"]==AGE1)|(data["AGE_GB"]==AGE2)|(data["AGE_GB"]==AGE3))]
    domestic=data1[data1["TOUR_GB"] == "도민"].index
    data2=data1.drop(domestic)
    data2=data2.iloc[:,2:]
    data3 =data2.groupby(["TA_YM","M_DONG_NM","AGE_GB",'IDCS_CD']).sum()
    data3=data3.reset_index()
    data3 =data3[["TA_YM","M_DONG_NM","AGE_GB",'IDCS_CD',"AMT_CORR"]]

    data4 =data3.groupby(['TA_YM','IDCS_CD']).sum()
    data4=data4.reset_index()
    for i in data4.iloc[:,1]:
        if i in G_R:
            data4["IDCS_CD"] = data4["IDCS_CD"].replace(i, "일반음식점")
        elif i in R_R:
            data4["IDCS_CD"] = data4["IDCS_CD"].replace(i, "기타음식점")
        elif i in A_F:
            data4["IDCS_CD"] = data4["IDCS_CD"].replace(i, "숙박시설")
        elif i in R_S:
            data4["IDCS_CD"] = data4["IDCS_CD"].replace(i, "소매업")
        elif i in Sport:
            data4["IDCS_CD"] = data4["IDCS_CD"].replace(i, "여가업")
        elif i in Rent_S:
            data4["IDCS_CD"] = data4["IDCS_CD"].replace(i, "임대업")    
    data4 =data4[['TA_YM','IDCS_CD','AMT_CORR']]
    
    na=data4[(data4["IDCS_CD"] != "일반음식점") & (data4["IDCS_CD"] != "기타음식점")&
        (data4["IDCS_CD"] != "숙박시설")&(data4["IDCS_CD"] != "소매업")&(data4["IDCS_CD"] != "임대업")&(data4["IDCS_CD"] != "여가업")].index
    data4=data4.drop(na)

    YM=data4["TA_YM"].drop_duplicates()
    IDCS=data4["IDCS_CD"].drop_duplicates()

    fin_CO=pd.DataFrame()
    fin_CO["연도"]=YM
    for word in IDCS:
        fin_R_S=data4[data4["IDCS_CD"]=="%s" %word].groupby(['TA_YM']).sum()
        fin_R_S=fin_R_S.reset_index()
    for year in YM:
        if year in fin_R_S["TA_YM"].values:
            pass
        elif year not in fin_R_S["TA_YM"].values:
            small_s = {year: (year,"%s" %word,0)}
            for col,ro in small_s.items():
                data4.loc[col] = ro
                data4=data4.sort_values(by=['TA_YM'])
                data4=data4.reset_index(drop=True)

    for a in IDCS:
        fin_CO.loc[:,"%s" %a]= data4[data4["IDCS_CD"]== "%s" %a].groupby(['TA_YM'])["AMT_CORR"].sum().values
    #for i,j in zip(list1,list2):
    #    fin_CO.loc[:,"%s"%i]=df4[df4["IDCS_CD"]== "%s"%j].groupby(['TA_YM'])["USECT_CORR"].sum().values
    fin_CO.to_csv('C:\\Users\\user\\Desktop\\%s.csv'%input,encoding='cp949')


industry(df,1,2,3,input="중문")